In [84]:
import pandas as pd
from basketball_reference_web_scraper.data import Team, TEAM_ABBREVIATIONS_TO_TEAM

In [56]:
team_mapping = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BRK': 'Brooklyn Nets',
    'CHI': 'Chicago Bulls',
    'CHO': 'Charlotte Hornets',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards',

    # DEPRECATED TEAMS
    'NJN': 'New Jersey Nets',
    'NOH': 'New Orleans Hornets',
    'NOK': 'New Orleans/Oklahoma City Hornets',
    'CHA': 'Charlotte Bobcats',
    'CHH': 'Charlotte Hornets',
    'SEA': 'Seattle Supersonics',
    'VAN': 'Vancouver Grizzlies',
}

In [57]:
team_mapping

{'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',
 'BRK': 'Brooklyn Nets',
 'CHI': 'Chicago Bulls',
 'CHO': 'Charlotte Hornets',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NOP': 'New Orleans Pelicans',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spurs',
 'TOR': 'Toronto Raptors',
 'UTA': 'Utah Jazz',
 'WAS': 'Washington Wizards',
 'NJN': 'New Jersey Nets',
 'NOH': 'New Orleans Hornets',
 'NOK': 'New Orleans/Oklahoma City Hornets',
 'CHA': 'Charlotte Bobcats',
 'CHH': 'Charlotte Hornets

In [58]:
basic_stats_path = 'data/not_raw_data/per_game_stats/basic_per_game_stats.csv'
bstats = pd.read_csv(basic_stats_path).drop(columns=['Unnamed: 0'])
bstats.loc[:, 'Player'] = bstats.Player.str.replace("*", "", regex=False)
bstats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,...,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,...,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,...,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,...,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [59]:
mvps_path = 'data/not_raw_data/mvp_voting/mvp_voting.csv'
mvps = pd.read_csv(mvps_path).drop(columns=['Unnamed: 0'])

mvps = mvps.drop(columns = mvps.columns[mvps.columns.isin(bstats.columns.values.tolist()) & (~mvps.columns.isin(['Player', 'Year']))])
mvps.loc[:, 'Rank'] = mvps.loc[:, 'Rank'].str.replace('T', '').astype('int64')

mvps.head()

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year
0,1,Michael Jordan,77.0,891.0,960,0.928,20.3,0.321,1991
1,2,Magic Johnson,10.0,497.0,960,0.518,15.4,0.251,1991
2,3,David Robinson,6.0,476.0,960,0.496,17.0,0.264,1991
3,4,Charles Barkley,2.0,222.0,960,0.231,13.4,0.258,1991
4,5,Karl Malone,0.0,142.0,960,0.148,15.5,0.225,1991


In [60]:
def single_row(df):
    if df.shape[0] == 1:
        return df
    else:
        row = df[df['Tm'] == 'TOT']
        row['Tm'] = df.iloc[-1, :]['Tm']
        return row

# Taking single row of basic year stats for each combination of player and year    
    
bstats = bstats.groupby(['Player', 'Year']).apply(single_row)
bstats.index = bstats.index.droplevel()
bstats.index = bstats.index.droplevel()
bstats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
164,135,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,...,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,1991
633,141,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,...,3.7,5.6,9.3,1.4,1.1,0.4,1.4,1.7,13.6,1992
1092,137,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,...,3.5,5.2,8.7,1.4,1.1,0.5,1.4,1.8,12.8,1993
1579,149,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,...,3.4,5.8,9.2,1.7,0.9,0.5,1.2,1.7,14.7,1994
2067,142,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,...,2.4,5.8,8.2,1.5,0.7,0.4,1.4,1.8,11.2,1995


In [61]:
advanced_stats_path = 'data/not_raw_data/per_game_stats/advanced_per_game_stats.csv'
astats = pd.read_csv(advanced_stats_path).drop(columns=['Unnamed: 0', 'age', 'positions', 'team', 'slug', 'is_combined_totals', 'games_played']).rename(columns={'name': 'Player'})
astats.head()

,assist_percentage,block_percentage,box_plus_minus,defensive_box_plus_minus,defensive_rebound_percentage,defensive_win_shares,free_throw_attempt_rate,minutes_played,Player,offensive_box_plus_minus,...,steal_percentage,three_point_attempt_rate,total_rebound_percentage,true_shooting_percentage,turnover_percentage,usage_percentage,value_over_replacement_player,win_shares,win_shares_per_48_minutes,Year
0,5.8,2.5,-4.6,-1.2,23.4,0.5,0.379,290,Alaa Abdelnaby,-3.4,...,0.7,0.000,17.0,0.499,14.0,22.1,-0.2,0.5,0.079,1991
1,19.2,0.1,-5.0,-3.0,6.0,-0.3,0.097,1505,Mahmoud Abdul-Rauf,-2.0,...,1.5,0.099,3.8,0.448,9.5,27.2,-1.1,-1.0,-0.031,1991
2,2.5,1.1,-3.0,-0.2,18.7,1.1,0.472,1313,Mark Acres,-2.8,...,0.9,0.014,14.9,0.551,14.0,9.3,-0.3,2.5,0.090,1991
3,39.4,0.1,5.3,-0.7,8.8,0.4,0.372,2346,Michael Adams,6.0,...,2.6,0.397,5.2,0.530,12.7,28.5,4.3,6.3,0.128,1991
4,11.6,0.6,1.4,0.2,13.7,2.7,0.349,2006,Mark Aguirre,1.2,...,1.2,0.086,10.7,0.526,10.9,25.7,1.7,5.5,0.132,1991


#### When a player was traded to another team midseason, they have two rows of advanced stats for each team they played on
#### Here I am taking the last row for each combintaion of player and year in order to ensure that the players' advanced stats from the second team and most recent span are used

In [62]:
def single_row(df):
    if df.shape[0] == 1:
        return df
    else:
        print(df.iloc[-1].index)
        return df.iloc[-1]

astats = astats.groupby(['Player', 'Year']).last()
astats = astats.reset_index()

In [63]:
standings_path = 'data/not_raw_data/team_standings/standings.csv'
standings = pd.read_csv(standings_path).drop(columns=['Unnamed: 0'])

# Removing the irrelevent rows
standings  = standings[~standings['Team'].isin(['Midwest Division', 'Central Division', 'Northwest Division', 'Pacific Division', 'Southeast Division','Southwest Division',])]

standings.head(10)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
5,24,58,.293,32.0,101.8,107.8,-5.91,1991,Miami Heat
7,61,21,.744,—,110.0,101.0,8.57,1991,Chicago Bulls
8,50,32,.610,11.0,100.1,96.8,3.08,1991,Detroit Pistons
9,48,34,.585,13.0,106.4,104.0,2.33,1991,Milwaukee Bucks
10,43,39,.524,18.0,109.8,109.0,0.72,1991,Atlanta Hawks


In [64]:
standings.Team.unique()

array(['Boston Celtics', 'Philadelphia 76ers', 'New York Knicks',
       'Washington Bullets', 'New Jersey Nets', 'Miami Heat',
       'Chicago Bulls', 'Detroit Pistons', 'Milwaukee Bucks',
       'Atlanta Hawks', 'Indiana Pacers', 'Cleveland Cavaliers',
       'Charlotte Hornets', 'San Antonio Spurs', 'Utah Jazz',
       'Houston Rockets', 'Orlando Magic', 'Minnesota Timberwolves',
       'Dallas Mavericks', 'Denver Nuggets', 'Portland Trail Blazers',
       'Los Angeles Lakers', 'Phoenix Suns', 'Golden State Warriors',
       'Seattle SuperSonics', 'Los Angeles Clippers', 'Sacramento Kings',
       'Toronto Raptors', 'Vancouver Grizzlies', 'Washington Wizards',
       'Memphis Grizzlies', 'New Orleans Hornets', 'Charlotte Bobcats',
       'New Orleans/Oklahoma City Hornets', 'Oklahoma City Thunder',
       'Brooklyn Nets', 'New Orleans Pelicans'], dtype=object)

### Merging the mvp voting details and the basic stats of the player 

In [65]:
df = mvps.merge(bstats, on=['Player', 'Year'], how='outer')
df.head()

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Michael Jordan,77.0,891.0,960.0,0.928,20.3,0.321,1991,175,...,.851,1.4,4.6,6.0,5.5,2.7,1.0,2.5,2.8,31.5
1,2.0,Magic Johnson,10.0,497.0,960.0,0.518,15.4,0.251,1991,171,...,.906,1.3,5.6,7.0,12.5,1.3,0.2,4.0,1.9,19.4
2,3.0,David Robinson,6.0,476.0,960.0,0.496,17.0,0.264,1991,287,...,.762,4.1,8.9,13.0,2.5,1.5,3.9,3.3,3.2,25.6
3,4.0,Charles Barkley,2.0,222.0,960.0,0.231,13.4,0.258,1991,21,...,.722,3.9,6.3,10.1,4.2,1.6,0.5,3.1,2.6,27.6
4,5.0,Karl Malone,0.0,142.0,960.0,0.148,15.5,0.225,1991,212,...,.770,2.9,8.9,11.8,3.3,1.1,1.0,3.0,3.3,29.0


### Next, merging advanced stats from players

In [66]:
df = df.merge(astats, on=['Player', 'Year'], how='inner')
df.head()

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,player_efficiency_rating,steal_percentage,three_point_attempt_rate,total_rebound_percentage,true_shooting_percentage,turnover_percentage,usage_percentage,value_over_replacement_player,win_shares,win_shares_per_48_minutes
0,1.0,Michael Jordan,77.0,891.0,960.0,0.928,20.3,0.321,1991,175,...,31.6,3.7,0.051,9.5,0.605,8.7,32.9,10.8,20.3,0.321
1,2.0,Magic Johnson,10.0,497.0,960.0,0.518,15.4,0.251,1991,171,...,25.1,1.8,0.256,10.9,0.623,20.4,22.9,8.1,15.4,0.251
2,3.0,David Robinson,6.0,476.0,960.0,0.496,17.0,0.264,1991,287,...,27.4,2.0,0.005,19.0,0.615,13.7,26.6,8.2,17.0,0.264
3,4.0,Charles Barkley,2.0,222.0,960.0,0.231,13.4,0.258,1991,21,...,28.9,2.2,0.133,15.3,0.635,12.6,29.1,7.0,13.4,0.258
4,5.0,Karl Malone,0.0,142.0,960.0,0.148,15.5,0.225,1991,212,...,24.8,1.4,0.009,17.2,0.596,10.9,30.1,6.2,15.5,0.225


In [67]:
# Column of rank only has values for players in the mvp race. Others are left NaN so we fill na's with 0
df.Rank = df.Rank.fillna(0, axis=0)

In [81]:
df

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,player_efficiency_rating,steal_percentage,three_point_attempt_rate,total_rebound_percentage,true_shooting_percentage,turnover_percentage,usage_percentage,value_over_replacement_player,win_shares,win_shares_per_48_minutes
0,1.0,Michael Jordan,77.0,891.0,960.0,0.928,20.3,0.321,1991,175,...,31.6,3.7,0.051,9.5,0.605,8.7,32.9,10.8,20.3,0.321
1,2.0,Magic Johnson,10.0,497.0,960.0,0.518,15.4,0.251,1991,171,...,25.1,1.8,0.256,10.9,0.623,20.4,22.9,8.1,15.4,0.251
2,3.0,David Robinson,6.0,476.0,960.0,0.496,17.0,0.264,1991,287,...,27.4,2.0,0.005,19.0,0.615,13.7,26.6,8.2,17.0,0.264
3,4.0,Charles Barkley,2.0,222.0,960.0,0.231,13.4,0.258,1991,21,...,28.9,2.2,0.133,15.3,0.635,12.6,29.1,7.0,13.4,0.258
4,5.0,Karl Malone,0.0,142.0,960.0,0.148,15.5,0.225,1991,212,...,24.8,1.4,0.009,17.2,0.596,10.9,30.1,6.2,15.5,0.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,0.0,Željko Rebrača,NaN,NaN,NaN,NaN,NaN,NaN,2002,336,...,16.2,1.3,0.000,14.5,0.569,15.7,21.4,-0.1,3.3,0.134
14693,0.0,Željko Rebrača,NaN,NaN,NaN,NaN,NaN,NaN,2003,316,...,14.0,0.7,0.000,11.2,0.596,14.9,19.2,-0.1,1.3,0.133
14694,0.0,Željko Rebrača,NaN,NaN,NaN,NaN,NaN,NaN,2004,338,...,14.1,0.0,0.000,10.0,0.523,14.3,24.9,0.0,0.0,0.016
14695,0.0,Željko Rebrača,NaN,NaN,NaN,NaN,NaN,NaN,2005,350,...,14.0,0.8,0.000,11.9,0.625,15.3,16.0,0.2,2.4,0.122


In [69]:
# Map team names
df['Tm'] = df['Tm'].map(team_mapping)

In [70]:
# Add standings of the team for each players record
final_df = df.merge(standings, left_on=['Tm', 'Year'], right_on=['Team', 'Year'])

In [83]:
final_df.loc[final_df.Rank.sort_values(ascending=False).index, :][120:180]

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,win_shares,win_shares_per_48_minutes,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
3397,11.0,Tim Duncan,0.0,2.0,1210.0,0.002,10.1,0.191,2009,120,...,10.1,0.191,54,28,.659,—,97.0,93.3,3.36,San Antonio Spurs
2324,11.0,Jamal Mashburn,0.0,1.0,1190.0,0.001,9.0,0.130,2003,240,...,9.0,0.130,47,35,.573,3.0,93.9,91.8,1.52,New Orleans Hornets
2670,11.0,Kevin Garnett,0.0,15.0,1270.0,0.012,16.1,0.248,2005,148,...,16.1,0.248,44,38,.537,8.0,96.8,95.3,1.73,Minnesota Timberwolves
2135,11.0,Paul Pierce,0.0,22.0,1260.0,0.017,12.9,0.187,2002,323,...,12.9,0.187,49,33,.598,3.0,96.4,94.1,1.75,Boston Celtics
3582,11.0,Manu Ginóbili,0.0,3.0,1230.0,0.002,9.7,0.216,2010,157,...,9.7,0.216,50,32,.610,5.0,101.4,96.3,5.07,San Antonio Spurs
4099,11.0,Stephen Curry,0.0,3.0,1210.0,0.002,11.2,0.180,2013,97,...,11.2,0.180,47,35,.573,9.0,101.2,100.3,1.32,Golden State Warriors
3945,11.0,Derrick Rose,0.0,5.0,1210.0,0.004,6.0,0.211,2012,372,...,6.0,0.211,50,16,.758,—,96.3,88.2,7.43,Chicago Bulls
266,11.0,Brad Daugherty,0.0,24.0,960.0,0.025,11.7,0.213,1992,85,...,11.7,0.213,57,25,.695,10.0,108.9,103.4,5.34,Cleveland Cavaliers
3040,11.0,Chauncey Billups,0.0,4.0,1290.0,0.003,11.4,0.216,2007,40,...,11.4,0.216,53,29,.646,—,96.0,91.8,3.69,Detroit Pistons
1937,11.0,Vince Carter,0.0,7.0,1240.0,0.006,12.9,0.208,2001,65,...,12.9,0.208,47,35,.573,5.0,97.6,95.4,1.69,Toronto Raptors


In [77]:
final_df[final_df.Tm == 'Los Angeles Lakers']

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,win_shares,win_shares_per_48_minutes,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
12,2.0,Magic Johnson,10.0,497.0,960.0,0.518,15.4,0.251,1991,171,...,15.4,0.251,58,24,.707,5.0,106.3,99.6,6.73,Los Angeles Lakers
13,0.0,A.C. Green,NaN,NaN,NaN,NaN,NaN,NaN,1991,135,...,6.1,0.135,58,24,.707,5.0,106.3,99.6,6.73,Los Angeles Lakers
14,0.0,Byron Scott,NaN,NaN,NaN,NaN,NaN,NaN,1991,301,...,6.6,0.120,58,24,.707,5.0,106.3,99.6,6.73,Los Angeles Lakers
15,0.0,Elden Campbell,NaN,NaN,NaN,NaN,NaN,NaN,1991,51,...,1.1,0.135,58,24,.707,5.0,106.3,99.6,6.73,Los Angeles Lakers
16,0.0,Irving Thomas,NaN,NaN,NaN,NaN,NaN,NaN,1991,330,...,-0.1,-0.057,58,24,.707,5.0,106.3,99.6,6.73,Los Angeles Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13724,0.0,Ryan Kelly,NaN,NaN,NaN,NaN,NaN,NaN,2014,251,...,2.2,0.081,27,55,.329,30.0,103.0,109.2,-5.33,Los Angeles Lakers
13725,0.0,Shawne Williams,NaN,NaN,NaN,NaN,NaN,NaN,2014,471,...,0.7,0.045,27,55,.329,30.0,103.0,109.2,-5.33,Los Angeles Lakers
13726,0.0,Steve Nash,NaN,NaN,NaN,NaN,NaN,NaN,2014,327,...,0.1,0.017,27,55,.329,30.0,103.0,109.2,-5.33,Los Angeles Lakers
13727,0.0,Wesley Johnson,NaN,NaN,NaN,NaN,NaN,NaN,2014,241,...,1.9,0.041,27,55,.329,30.0,103.0,109.2,-5.33,Los Angeles Lakers


In [24]:
final_df

,Rank,Player,First,Pts Won,Pts Max,Share,WS,WS/48,Year,Rk,...,win_shares,win_shares_per_48_minutes,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
0,1.0,Michael Jordan,77.0,891.0,960.0,0.928,20.3,0.321,1991,175,...,20.3,0.321,61,21,.744,—,110.0,101.0,8.57,Chicago Bulls
1,NaN,B.J. Armstrong,0.0,0.0,0.0,0.000,0.0,0.000,1991,14,...,4.1,0.114,61,21,.744,—,110.0,101.0,8.57,Chicago Bulls
2,NaN,Bill Cartwright,0.0,0.0,0.0,0.000,0.0,0.000,1991,55,...,4.2,0.089,61,21,.744,—,110.0,101.0,8.57,Chicago Bulls
3,NaN,Cliff Levingston,0.0,0.0,0.0,0.000,0.0,0.000,1991,200,...,2.0,0.095,61,21,.744,—,110.0,101.0,8.57,Chicago Bulls
4,NaN,Craig Hodges,0.0,0.0,0.0,0.000,0.0,0.000,1991,157,...,1.7,0.099,61,21,.744,—,110.0,101.0,8.57,Chicago Bulls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14315,NaN,Mark Hendrickson,0.0,0.0,0.0,0.000,0.0,0.000,2000,173,...,0.0,0.030,31,51,.378,21.0,98.0,99.0,-1.18,New Jersey Nets
14316,NaN,Michael Cage,0.0,0.0,0.0,0.000,0.0,0.000,2000,64,...,0.7,0.140,31,51,.378,21.0,98.0,99.0,-1.18,New Jersey Nets
14317,NaN,Scott Burrell,0.0,0.0,0.0,0.000,0.0,0.000,2000,62,...,3.2,0.115,31,51,.378,21.0,98.0,99.0,-1.18,New Jersey Nets
14318,NaN,Sherman Douglas,0.0,0.0,0.0,0.000,0.0,0.000,2000,114,...,0.6,0.100,31,51,.378,21.0,98.0,99.0,-1.18,New Jersey Nets


In [23]:
final_df.isna().sum()

Rank       13848
Player         0
First          0
Pts Won        0
Pts Max        0
           ...  
GB             0
PS/G           0
PA/G           0
SRS            0
Team           0
Length: 67, dtype: int64

In [85]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 3008-1C78

 Directory of C:\Users\rivverg\DontBeABitch\Projects\my-shit\basketball-betting-ml\src

10/31/2022  12:50 AM    <DIR>          .
04/03/2022  12:45 PM    <DIR>          ..
04/05/2022  10:21 PM             1,928 .gitignore
10/30/2022  11:46 PM    <DIR>          .idea
09/05/2022  08:30 AM    <DIR>          .ipynb_checkpoints
09/13/2022  08:06 PM    <DIR>          __pycache__
04/05/2022  06:36 PM                 0 basketball_rerference_api_examples.py
09/13/2022  08:15 PM            69,494 bball_reference_api_examples.ipynb
08/31/2022  09:57 PM    <DIR>          data
07/11/2022  06:09 PM             9,044 data.py
08/31/2022  09:57 PM    <DIR>          dependencies
07/11/2022  07:51 PM             4,920 dev.py
09/13/2022  08:17 PM             4,839 develop.py
07/17/2022  08:01 AM            53,024 developing.ipynb
04/26/2022  10:28 PM             5,110 extract.ipynb
05/02/2022  11:11 PM             2,117 extract.py
08/31/2022

In [86]:
final_df.to_csv("data/not_raw_data/player_data.csv")